In [58]:
import time
import requests
from bs4 import BeautifulSoup
import json 
import random
import ipywidgets as widgets
from IPython.display import HTML, display, clear_output
from ipywidgets import FloatProgress

In [59]:
equation_dict = {"17-8": "9", "11+22": "33" , "5+8+6" : "19", "3+5+10" : "18", "21-2+3" : "22",
                 "28+67" : "95" , "55-42": "13", "95-67" : "28", "52-20+10" : "42", "65+19-15" : "69",
                 "5x6+15" : "45", "8x6-15" : "33", "3x9+28" : "55", "29x3-18" : "69", "42-7x3" : "21" }
#prepare dictionary to match question with answer

questions_list = [["17", "-8"], ["11", "+22"], ["5", "+8", "+6"], ["3", "+5", "+10"], ["21", "-2", "+3"], 
                  ["28","+67"],["55","-42"], ["95","-67"], ["52","-20","+10"],["65","+19","-15"] , 
                  ["5","x6","+15"],["8","x6","-15"],["3","x9","+28"],["29","x3","-18"], ["42","-7","x3"]]
#questions list with increasing levels of difficulty

In [60]:
def send_to_google_form(data_dict, form_url):
    ''' 
        Helper function to upload information to a corresponding google form.

    Args:
        data_dict (dict): A dictionary containing data to be uploaded to the form. 
            Keys should correspond to the field names in the Google Form.
        form_url (str): The URL of the Google Form to which the data will be uploaded.

    Returns:
        bool: True if the data is successfully uploaded to the form, False otherwise.

    This function extracts the form ID from the provided form URL, constructs the view form URL 
    and the post form URL, and then submits the data to the Google Form. It checks if all the 
    required fields in the form are present in the data dictionary before uploading.

    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [61]:
data_dict={
        'consent' :[],
        'name' : [],
        'age' : [],
        'gender': [],
        'score' : [],
        'average_all_time': [],
        'average_level_1' : [],
        'average_level_2': [],
        'average_level_3': []
        
        
        
    }
#dictionary to store items that will be saved in a google form for collecting user data

In [62]:
def the_starting_page():
    '''
    Function to display questions before the test begins. The user will be asked their name, age, gender, 
    and consent to use their data for educational teaching purposes.

    This function prompts the user for their name, age (accepts only integer input), gender, and consent 
    to use their data for educational purposes. It then displays information regarding data usage and 
    asks for consent. After receiving the user's responses, it prints a welcome message for the math test.
    
    Returns:
        None

    '''
    print("What is your name?")
    name = input(">")
    print("What is your age?")
        
    while True:
        age = input("Please enter a number ")
        try:
            val = int(age)
            break;
        except ValueError:
            print("This is not a number. Please enter a valid number")

    #ask user for age which will only accept integer input
        
    print("What is your gender")
    gender = input(">")
    clear_output()



    print("Please read:")
    print("")
    print("we wish to record your response data")
    print("to an anonymised public data repository. ")
    print("Your data will be used for educational teaching purposes")
    print("practising data analysis and visualisation.")
    print("")
    print("Please type   yes   in the box below if you consent to the upload.")
    consent = input("> ")
    consent = consent.lower()
    
    if consent == "yes":
        print("Thanks - your data will be uploaded.")
 
    else:
        print("No problem we hope you enjoyed the test.")

        
    print("Welcome! This is a math test")
    time.sleep(2)
    clear_output()
    time.sleep(2)
    print("Goodluck")
    time.sleep(2)
    clear_output()

    data_dict['consent'].append(consent)
    data_dict['name'].append(name)
    data_dict['age'].append(age)
    data_dict['gender'].append(gender)
 
        
    return

In [63]:
def math_questions(questions_list):
    '''

    Function to run a math test using questions from a provided list.

    Args:
        questions_list (list): A list containing math questions. Each question should be represented as a list 
            of strings, where each string represents a part of the equation.

    Returns:
        tuple: A tuple containing two elements:
            - A list of floats representing the time taken to answer each question.
            - An integer representing the score earned in the test (1 point for each correct answer).

    This function runs a math test using the questions provided in the `questions_list`. It displays each question 
    one at a time, alongside a progress bar. The user is prompted to input their answer for each question. 
    The function evaluates the user's answer against the correct answer stored in `equation_dict` and updates 
    the score accordingly. It also records the time taken to answer each question and returns it along with the score.
    '''
    
    score = 0
    all_time = []
    #creates empty list, all_time ,to keep time taken to answer each question

    progress_bar = FloatProgress(min=0, max=len(questions_list), description='Progress:')
    display(progress_bar)
    #code to display progress bar from  ipywidget. Maximum is setup to be equal to number of questions in test 
    
    for question_index, questions in enumerate(questions_list, start=1):
        equation= ""
        color = "black"
    
        
        style = f"color: {color}; font-size: 50px; ;"
        html_out = HTML(f"""<span style='{style}'>Question {question_index}</span>""")
        display(html_out)
        time.sleep(2)
        clear_output()

        #iterates through questions in question_list and displasy corresponding question number

        for step in questions:
            style = f"color: {color}; font-size: 50px; ;"
            html_out = HTML(f"""<span style='{style}'>{step}</span>""")
            display(html_out)
            time.sleep(2)
            clear_output()

            equation += step

        #display each part of the equation seperately one at a time 
        
        start_time = time.time()
        answer = input("Answer:")
        end_time = time.time()
        
        #measures time taken to answer each question
        
        correct_answer = equation_dict[equation]


        style = f"color: {color}; font-size: 50px; ;"
        html_out = HTML(f"""<span style='{style}'>{correct_answer}</span>""")
        display(html_out)
        time.sleep(2)
        clear_output()
        
        if correct_answer == answer:
            score += 1
        else:
            score = score
        #evaluates user's answer and gives 1 pont if user's answer matches answer for equation in equation_dict
        
        time_taken = end_time - start_time
        all_time.append(time_taken)
        
        
        progress_bar.value += 1  
        display(progress_bar)
        #progress bar will increase in increaments after each question

    return all_time, score
          

In [64]:
def math_test(questions_list):
    """ 
    Function to run the entire math test.

    Args:
        questions_list (list): A list containing math questions. Each question should be represented as a list 
            of strings, where each string represents a part of the equation.

    Returns:
        None

    This function starts the math test by displaying introductory questions using `the_starting_page()` function. 
    It then proceeds to run the test using the questions provided in `questions_list`, recording the time taken 
    to answer each question and the score earned. After the test is completed, it calculates the average time taken 
    to answer questions for each difficulty level and overall. The function displays the average time and final score 
    to the user. Additionally, it sends the test results to a Google Form specified by `form_url`.
    
    """
    the_starting_page()

    all_time, score = math_questions(questions_list)

    level_1_time = all_time[:4]
    average_level_1 = sum(level_1_time) / len(level_1_time)

    level_2_time = all_time[5:9]
    average_level_2 = sum(level_2_time) / len(level_2_time)

    level_3_time = all_time[10:14]
    average_level_3 = sum(level_3_time) / len(level_3_time)
    
    average_time = sum(all_time) / 15
    
    #Finds the average time taken to answer each question overall and average time taken to answer each question for each level
    
    average_time_rounded = round(average_time,3)
    message = f"Average time: {average_time_rounded} seconds \n , Score: {score}"
    
    style = f"color: black; font-size: 20px; ;"
    html_out = HTML(f"""<span style='{style}'>{message}</span>""")
    display(html_out)

    #displays average time taken to answer each question overall and total score 

    data_dict['score'].append(score)
    data_dict['average_all_time'].append(average_time_rounded)
    data_dict['average_level_1'].append(average_level_1)
    data_dict['average_level_2'].append(average_level_2)
    data_dict['average_level_3'].append(average_level_3)
   
    
    
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSfYbmNMuv0gS-z0Ym1DosIVpEmb62AKXHSBRWFmMj6lU3S7eA/viewform?usp=sf_link'
    send_to_google_form(data_dict, form_url)
    
   #sends item in data_dict to google form 

    return

In [65]:
#full run of the test

math_test(questions_list)

KeyboardInterrupt: Interrupted by user